---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [7]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [8]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [9]:
import pandas as pd 
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
     
    city_home = pd.read_csv('City_Zhvi_AllHomes.csv')
    gdplev = pd.read_excel('gdplev.xls',skiprows=7)
    uni_towns = open('university_towns.txt', 'r') 
    city_home
    s =uni_towns.read().split('\n')
    key = 0
    lst = []
    for i in range(len(s)):
        if 'edit' in s[i]:
            s[i] = s[i].split('[')[0]
            key = i  
        if key == i:
            pass;
        else:
            lst.append([s[key], s[i].split('(')[0].strip()])  
    df=pd.DataFrame(lst, columns=["State", "RegionName"])
    df=df.drop([517],axis=0)
    return df

get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [20]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    gdplev = pd.read_excel('gdplev.xls',skiprows=7)
    gdplev = gdplev[['Unnamed: 4','Unnamed: 6']]
    gdplev[gdplev['Unnamed: 4'] == '2000q1']
    gdplev = gdplev.iloc[212:,:]
    gdplev = gdplev.reset_index(drop=True)
    gdplev = gdplev.rename(columns={'Unnamed: 4':'Q','Unnamed: 6':'GDP'})
    reccession = 0
    lst = []
    for quarter in range(1,len(gdplev.head(100))):
        prev_quarter = quarter-1
        if gdplev['GDP'][quarter] < gdplev['GDP'][prev_quarter]:
            if reccession != 1:
                reccession = 1
            else:
                lst.append(gdplev['Q'][quarter-1])
        else:
            if reccession == 1:
                reccession = 0

    return lst[0]
get_recession_start()

'2008q3'

In [26]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    gdplev = pd.read_excel('gdplev.xls',skiprows=7)
    gdplev = gdplev[['Unnamed: 4','Unnamed: 6']]
    gdplev[gdplev['Unnamed: 4'] == '2000q1']
    gdplev = gdplev.iloc[212:,:]
    gdplev = gdplev.reset_index(drop=True)
    gdplev = gdplev.rename(columns={'Unnamed: 4':'Q','Unnamed: 6':'GDP'})
    reccession = 0
    lst = []
    for quarter in range(1,len(gdplev.head(100))):
        prev_quarter = quarter-1
        if gdplev['GDP'][quarter] < gdplev['GDP'][prev_quarter]:
            if reccession != 1:
                reccession = 1
            else:
                lst.append(gdplev['Q'][quarter-1])
        else:
            if reccession == 1:
                reccession = 0  
    i = gdplev[gdplev['Q'] == lst[-1]].index

    #Add the two comming bull market
    lst.append(gdplev['Q'][i[0]+1])
    lst.append(gdplev['Q'][i[0]+2])
        
    return lst[-1]
get_recession_end()

'2009q3'

In [28]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    gdplev = pd.read_excel('gdplev.xls',skiprows=7)
    gdplev = gdplev[['Unnamed: 4','Unnamed: 6']]
    gdplev[gdplev['Unnamed: 4'] == '2000q1']
    gdplev = gdplev.iloc[212:,:]
    gdplev = gdplev.reset_index(drop=True)
    gdplev = gdplev.rename(columns={'Unnamed: 4':'Q','Unnamed: 6':'GDP'})
    reccession = 0
    lst = []
    for quarter in range(1,len(gdplev.head(100))):
        prev_quarter = quarter-1
        if gdplev['GDP'][quarter] < gdplev['GDP'][prev_quarter]:
            if reccession != 1:
                reccession = 1
            else:
                lst.append(gdplev['Q'][quarter-1])
        else:
            if reccession == 1:
                reccession = 0  
    i = gdplev[gdplev['Q'] == lst[-1]].index

    #Add the two comming bull market
    lst.append(gdplev['Q'][i[0]+1])
    lst.append(gdplev['Q'][i[0]+2])
    start = gdplev[gdplev['Q'] == lst[0]].index[0]
    end = gdplev[gdplev['Q'] == lst[0-1]].index[0]
    
    return gdplev[gdplev['GDP'] == gdplev.iloc[start:end]['GDP'].min()]['Q'].values[0]
get_recession_bottom()

'2009q2'

In [ ]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    city_home = pd.read_csv('City_Zhvi_AllHomes.csv')
    uni_towns = open('university_towns.txt', 'r')
    gdplev = pd.read_excel('gdplev.xls',skiprows=7)
    city_home_raw = city_home.drop(city_home.iloc[:,6:51], axis =1)
    city_home_raw
    states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 
              'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 
              'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont',
              'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin',
              'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 
              'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 
              'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana',
              'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado',
              'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands',
              'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

    #Change the state name 
    for i in range(len(city_home_raw)):
        if city_home_raw.iloc[i,2] in states:
            name = city_home_raw.iloc[i,2]
            city_home_raw.iloc[i,2] = city_home_raw.iloc[i,2].replace(name, states[city_home_raw.iloc[i,2]])

    #Set index name
    city_home_raw.set_index(['State','RegionName'],inplace=True)
    city_home_updated = city_home_raw

    dic ={}
    city_home_raw
    time_cols_df = city_home_updated.iloc[:,4:]
    dic = pd.to_datetime(time_cols_df.columns)
    #time_cols_df
    #time_cols_df_trans = time_cols_df.transpose()
    time_cols_df.columns = dic
    time_cols_df_trans = time_cols_df.transpose()
    time_cols_df_trans = time_cols_df_trans.resample('Q').mean()
    time_cols_df_trans.transpose()

    lst = []
    for i in range (2000,2017,1):
        for j in range(4):
            lst.append('{}q{}'.format(i,j+1))
    lst = lst[0:-1]
    time_cols_df_trans = time_cols_df_trans.transpose()
    time_cols_df_trans.columns = lst
    time_cols_df_trans
    return time_cols_df_trans
convert_housing_data_to_quarters()

In [39]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
   
    from scipy import stats
    
    lst_recession = ['2008q3', '2008q4', '2009q1', '2009q2', '2009q3']
    housing_data = convert_housing_data_to_quarters()
    housing_data = housing_data[lst_recession]
    housing_data = housing_data.reset_index()
    housing_data = housing_data[['RegionName','State','2008q3','2008q4','2009q1','2009q2','2009q3']]

    uni_towns = get_list_of_university_towns()
    uni_towns = uni_towns.set_index('RegionName')
    uni_towns = uni_towns.reset_index()

    uni_towns =  housing_data.merge(uni_towns,on=['RegionName','State'],how='inner')
    non_uni_towns = housing_data.merge(uni_towns,on=['RegionName','State'],how='left', indicator=True)
    non_uni_towns = non_uni_towns[non_uni_towns['_merge'] == 'left_only'].drop(columns=['_merge'])
    non_uni_towns =non_uni_towns.drop(columns=['2008q3_y','2008q4_y','2009q1_y','2009q2_y','2009q3_y'])
    non_uni_towns = non_uni_towns.rename(columns= {'2008q3_x':'2008q3','2008q4_x':'2008q4', '2009q1_x':'2009q1' ,'2009q2_x': '2009q2','2009q3_x':'2009q3'})


    ratio_uni_towns = uni_towns['2008q3']/uni_towns['2009q3']
    ratio_non_uni_towns = non_uni_towns['2008q3']/non_uni_towns['2009q3']

    t_stat , p = stats.ttest_ind(ratio_non_uni_towns,ratio_uni_towns, nan_policy = 'omit')

    if p<0.01:
        ans = "university town"
        different = True
    else:
        ans = "no university town"
        different = False
    
    return (different,p,ans )

run_ttest()

(True, 0.002238259417427338, 'university town')